In [25]:
# https://github.com/adeshpande3/LSTM-Sentiment-Analysis

import spacy
nlp = spacy.load('en')

In [35]:
inp = open('./data/imdb_master.txt', 'r')
out = open('./data/imdb_master_lemma.txt', 'w')

limit, count, t,sentifile, temp_t = 1, 0, None, '', None

for line in inp:
    #count += 1
    #if limit < count: break
    t = line[-4:-1]
    review = line[:-5]
    doc = nlp(review)
    if t != temp_t: print(f'DEBUT DOCUMENT {t}', file=out)
    temp_t = t
    for sent in doc.sents:
        print('DEBUT PHRASE', file=out)
        for token in sent:
            l = (token.i, token.text, token.lemma_, token.tag_, token.head.i, token.dep_)
            s = '\t'.join([str(e) for e in l])
            print(f'{s}', file=out)
            #print(s)
        print('FIN PHRASE')
    print('FIN DOCUMENT')

inp.close()
out.close()

In [3]:
import os, time, pickle, re

In [7]:
sentifile = open('./data/SentiWordNet_3.0.0.txt', 'r')

re_sentiline = re.compile(r'^([avn])\t([\d]+)\t([\d\.]+)\t([\d\.]+)\t([^\t]+)\t')
re_sentimots = re.compile(r'([a-z-]+)#([0-9]+)')

SENTI = {}
limit, count = 100, 0
for line in sentifile:
    res = re_sentiline.match(line)
    count += 1
    #if not line[0] == 'a': continue
    #if limit < count: break
    if not res: continue    
    pos = float(res.group(3))
    neg = float(res.group(4))
    res2 = re_sentimots.findall(res.group(5))
    for m in res2:
        if m[1] == '1':
            if pos == 0 and neg == 0:
                SENTI[m[0]] = (0, 0, 0)
            else :
                SENTI[m[0]] = (pos, neg, pos/(pos+neg))
                
out=open("./data/SentiWordNet.pickle","wb")
pickle.dump(SENTI,out)
out.close()

In [67]:
ilnp = open('./data/imdb_master_lemma.txt')
picklefile = open("./data/SentiWordNet.pickle","rb")
SENTI = pickle.load(picklefile)
picklefile.close()

re_lemma = re.compile(r"^([\d]+)\s([\w\.\,\'-?\(\)!\"`%&]+)\s([\w\.\,\-\'-?\(\)!\"`%& ]+)\s([\w\.\,$:\-\'`_]+)\s([\d]+)\s([\w]+)")
limit, count, ty, temp_ty = 1000, 0, None, None
corpus_val, corpus_class, a = list(), list(), list()
for line in ilnp:
    count += 1
    #if limit < count: break
    m = re.match(r'DEBUT DOCUMENT ([\w]+)', line)
    if m : 
        ty = m.group(1)
        a = []
        continue
    m = re.match(r'DEBUT PHRASE', line)
    if m: continue
    m = re.match(r'FIN PHRASE', line)
    if m: continue
    m = re.match(r'FIN DOCUMENT', line)
    if m: 
        if (len(a)>0):
            avg=sum(a)/len(a)
        else:
            avg=0
            corpus_val.append([avg])
            corpus_class.append(ty)
    m = re_lemma.match(line)
    if not m: continue
    lemma = m.group(3)
    if lemma in SENTI: 
        a.append(SENTI[lemma][2])
        # making an error => The appending should happen by review/sentiment, not by avery single found lemma
        #corpus_val.append(SENTI[lemma][2]) # sentiment append
        #corpus_class.append(ty)        
ilnp.close()

In [21]:
from sklearn.utils import shuffle
import numpy as np
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [61]:
def oneHot(l, class_assign):
    cl = np.unique(corpus_class)
    d = { c: ca for i, (c, ca) in enumerate(zip(cl, class_assign)) }
    return [ d[c] for c in l ]

In [70]:
corpus_class_oh = oneHot(corpus_class, [-1, 1])
corpus_val, corpus_class_oh = shuffle(corpus_val, corpus_class_oh)

In [71]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, corpus_val, corpus_class_oh, cv=5, n_jobs=5)
print("Two-classes accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Two-classes accuracy: 0.50 (+/- 0.00)


In [73]:
ilnp = open('./data/imdb_master_lemma.txt')
picklefile = open("./data/SentiWordNet.pickle","rb")
SENTI = pickle.load(picklefile)
picklefile.close()

re_lemma = re.compile(r"^([\d]+)\s([\w\.\,\'-?\(\)!\"`%&]+)\s([\w\.\,\-\'-?\(\)!\"`%& ]+)\s([\w\.\,$:\-\'`_]+)\s([\d]+)\s([\w]+)")
limit, count, ty, temp_ty = 1000, 0, None, None
corpus_val, corpus_class, a = list(), list(), list()
for line in ilnp:
    count += 1
    #if limit < count: break
    m = re.match(r'DEBUT DOCUMENT ([\w]+)', line)
    if m : 
        ty = m.group(1)
        a = []
        continue
    m = re.match(r'DEBUT PHRASE', line)
    if m: continue
    m = re.match(r'FIN PHRASE', line)
    if m: continue
    m = re.match(r'FIN DOCUMENT', line)
    if m: 
        if (len(a)>0):
            avg=sum(a)/len(a)
        else:
            avg=0
            corpus_val.append([avg])
            corpus_class.append(ty)
    m = re_lemma.match(line)
    if not m: continue
    lemma, depnat = m.group(3), m.group(5)
    if lemma in SENTI and depnat=='ROOT': 
        a.append(SENTI[lemma][2])  
ilnp.close()

In [74]:
corpus_class_oh = oneHot(corpus_class, [-1, 1])
corpus_val, corpus_class_oh = shuffle(corpus_val, corpus_class_oh)
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, corpus_val, corpus_class_oh, cv=5, n_jobs=5)
print("Two-classes accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Two-classes accuracy: 0.50 (+/- 0.00)
